In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'torchvision'

In [6]:
import torchaudio

ModuleNotFoundError: No module named 'torchaudio'

In [7]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(nz, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input.view(-1, 28*28))


In [ ]:
# Hyperparameters
nz = 100  # Size of z latent vector (i.e. size of generator input)
lr = 0.0002  # Learning rate
batch_size = 128  # Batch size
epochs = 100  # Number of training epochs

# Initialize the networks
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

# Optimizers
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCELoss()

# Fixed noise for generating images for visualization
fixed_noise = torch.randn(64, nz, device=device)


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Training loop
for epoch in range(epochs):
    for i, (data, _) in enumerate(dataloader):
        # Update Discriminator
        netD.zero_grad()
        real_data = data.to(device)
        batch_size = real_data.size(0)
        label = torch.full((batch_size,), 1., dtype=torch.float, device=device)
        
        output = netD(real_data).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        
        noise = torch.randn(batch_size, nz, device=device)
        fake_data = netG(noise)
        label.fill_(0.)
        
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        optimizerD.step()
        
        # Update Generator
        netG.zero_grad()
        label.fill_(1.)  # Reuse labels for generator loss
        
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()
        
        # Print and save losses
        if i % 100 == 0:
            print(f'[{epoch}/{epochs}][{i}/{len(dataloader)}] Loss_D: {errD_real.item() + errD_fake.item()} Loss_G: {errG.item()}')

    # Generate images for visualization and save
    if epoch % 10 == 0:
        with torch.no_grad():
            fake_images = netG(fixed_noise).detach().cpu()
        
        # Denormalize images
        fake_images = fake_images * 0.5 + 0.5
        
        # Save images
        vutils.save_image(fake_images, f'gan_images_epoch_{epoch}.png', nrow=8, normalize=True)

print("Training finished!")


In [ ]:

def plot_generated_images(netG, num_images=64):
    """
    Plots the output of the generator.

    Args:
        netG: The generator network.
        num_images: Number of images to generate and plot.
    """
    with torch.no_grad():
        noise = torch.randn(num_images, nz, device=device)
        fake_images = netG(noise).detach().cpu()

    # Denormalize images
    fake_images = fake_images * 0.5 + 0.5

    # Plot images
    fig, axs = plt.subplots(8, 8, figsize=(8, 8))
    for i, ax in enumerate(axs.flat):
        ax.imshow(fake_images[i].squeeze(), cmap='gray')
        ax.axis('off')

    plt.show()
